# 第七章：使用示例（少样本提示）

- [课程](#lesson)
- [练习](#exercises)
- [示例演练场](#example-playground)

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## Lesson

**Giving Claude examples of how you want it to behave (or how you want it not to behave) is extremely effective** for:
- 获得正确的答案
- 以正确的格式获得答案

这种提示方式也被称为"**少样本提示**"。您可能还会遇到"零样本"、"n样本"或"单样本"这些词。"样本"的数量指的是提示中使用了多少个示例。

### 示例

假设您是一名开发者，试图构建一个回答孩子问题的"家长机器人"。**Claude的默认响应相当正式和机械化**。这会让孩子很失望。

In [ ]:
# 提示
# 这个例子展示Claude默认的正式、机械化响应
PROMPT = "Will Santa bring me presents on Christmas?"

# 打印Claude的响应
print(get_completion(PROMPT))

您可以花时间描述您想要的语调，但**只给Claude一些理想响应的例子**会更容易。

In [ ]:
# 提示
# 这个改进版本通过提供理想响应的示例来训练Claude的语调
# 注意示例中温暖、亲切的"家长"语调
PROMPT = """Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?"""

# 打印Claude的响应
print(get_completion(PROMPT))

在下面的格式化示例中，我们可以逐步引导Claude完成一系列格式化指令，说明如何提取姓名和职业，然后按我们想要的方式格式化，或者我们可以**只为Claude提供一些正确格式化的示例，Claude可以从中推断**。注意`assistant`轮次中的`<individuals>`标签，这让Claude有一个好的开始。

In [ ]:
# 带有可变内容占位符的提示模板
# 这个复杂示例展示如何通过少样本提示让Claude学习特定的格式化模式
# 前两个段落提供了正确格式化的示例，第三个段落让Claude应用学到的模式
PROMPT = """Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.
<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MISICIAN AND COMPOSER]
4. Isabella Torres [CHEF]
</individuals>

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.
Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.
As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.
Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.
<individuals>
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]
</individuals>

Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town."""

# Claude响应的预填充
# 使用<individuals>标签引导Claude按照示例模式格式化输出
PREFILL = "<individuals>"

# 打印Claude的响应
print("--------------------------- 变量替换后的完整提示 ---------------------------")
print("用户轮次:")
print(PROMPT)
print("\n助手轮次:")
print(PREFILL)
print("\n------------------------------------- Claude的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## 练习
- [练习 7.1 - 通过示例进行电子邮件格式化](#exercise-71---email-formatting-via-examples)

### 练习 7.1 - 通过示例进行电子邮件格式化
我们将重做练习6.2，但这次我们将编辑`PROMPT`，使用电子邮件的"少样本"示例+正确分类（和格式化）来让Claude输出正确答案。我们希望Claude输出的*最后*一个字母是类别的字母。

如果您忘记了每封邮件对应的正确字母类别，请参考`EMAILS`列表中每封邮件旁边的注释。

记住这些是电子邮件的类别：										
- (A) 售前问题
- (B) 损坏或有缺陷的商品
- (C) 账单问题
- (D) 其他（请说明）								

In [ ]:
# 带有可变内容占位符的提示模板
# 需要修改这个提示来包含少样本示例，展示正确的分类格式
PROMPT = """Please classify this email as either green or blue: {email}"""

# Claude响应的预填充
PREFILL = ""

# 存储为列表的可变内容
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) 损坏或有缺陷的商品
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) 售前问题 或 (D) 其他（请说明）
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) 账单问题
    "How did I get here I am not good with computer.  Halp." # (D) 其他（请说明）
]

# 正确分类存储为列表的列表，以适应每封邮件可能有多个正确分类的情况
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 遍历电子邮件列表
for i,email in enumerate(EMAILS):
    
    # 将电子邮件文本替换到电子邮件占位符变量中
    formatted_prompt = PROMPT.format(email=email)
   
    # 获取Claude的响应
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # 对Claude的响应进行评分
    # 检查响应的最后一个字符是否为正确的类别字母
    grade = any([bool(re.search(ans, response[-1])) for ans in ANSWERS[i]])
    
    # 打印Claude的响应
    print("--------------------------- 变量替换后的完整提示 ---------------------------")
    print("用户轮次")
    print(formatted_prompt)
    print("\n助手轮次")
    print(PREFILL)
    print("\n------------------------------------- Claude的响应 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 评分 ------------------------------------------")
    print("本练习已正确解决:", grade, "\n\n\n\n\n\n")

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_7_1_hint; print(exercise_7_1_hint)

Still stuck? Run the cell below for an example solution.

In [ ]:
from hints import exercise_7_1_solution; print(exercise_7_1_solution)

### 恭喜！

如果您已经解决了到目前为止的所有练习，那么您已经准备好进入下一章了。祝您提示工程愉快！

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Prompt
PROMPT = "Will Santa bring me presents on Christmas?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = """Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.
Q: Will Santa bring me presents on Christmas?"""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Silvermist Hollow, a charming village, was home to an extraordinary group of individuals.
Among them was Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques at the regional medical center.
Olivia Chen was an innovative architect who transformed the village's landscape with her sustainable and breathtaking designs.
The local theater was graced by the enchanting symphonies of Ethan Kovacs, a professionally-trained musician and composer.
Isabella Torres, a self-taught chef with a passion for locally sourced ingredients, created a culinary sensation with her farm-to-table restaurant, which became a must-visit destination for food lovers.
These remarkable individuals, each with their distinct talents, contributed to the vibrant tapestry of life in Silvermist Hollow.
<individuals>
1. Dr. Liam Patel [NEUROSURGEON]
2. Olivia Chen [ARCHITECT]
3. Ethan Kovacs [MISICIAN AND COMPOSER]
4. Isabella Torres [CHEF]
</individuals>

At the heart of the town, Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant, Green Plate. Oliver's dedication to sourcing local, organic ingredients has earned the establishment rave reviews from food critics and locals alike.
Just down the street, you'll find the Riverside Grove Library, where head librarian Elizabeth Chen has worked diligently to create a welcoming and inclusive space for all. Her efforts to expand the library's offerings and establish reading programs for children have had a significant impact on the town's literacy rates.
As you stroll through the charming town square, you'll be captivated by the beautiful murals adorning the walls. These masterpieces are the work of renowned artist, Isabella Torres, whose talent for capturing the essence of Riverside Grove has brought the town to life.
Riverside Grove's athletic achievements are also worth noting, thanks to former Olympic swimmer-turned-coach, Marcus Jenkins. Marcus has used his experience and passion to train the town's youth, leading the Riverside Grove Swim Team to several regional championships.
<individuals>
1. Oliver Hamilton [CHEF]
2. Elizabeth Chen [LIBRARIAN]
3. Isabella Torres [ARTIST]
4. Marcus Jenkins [COACH]
</individuals>

Oak Valley, a charming small town, is home to a remarkable trio of individuals whose skills and dedication have left a lasting impact on the community.
At the town's bustling farmer's market, you'll find Laura Simmons, a passionate organic farmer known for her delicious and sustainably grown produce. Her dedication to promoting healthy eating has inspired the town to embrace a more eco-conscious lifestyle.
In Oak Valley's community center, Kevin Alvarez, a skilled dance instructor, has brought the joy of movement to people of all ages. His inclusive dance classes have fostered a sense of unity and self-expression among residents, enriching the local arts scene.
Lastly, Rachel O'Connor, a tireless volunteer, dedicates her time to various charitable initiatives. Her commitment to improving the lives of others has been instrumental in creating a strong sense of community within Oak Valley.
Through their unique talents and unwavering dedication, Laura, Kevin, and Rachel have woven themselves into the fabric of Oak Valley, helping to create a vibrant and thriving small town."""

# Prefill for Claude's response
PREFILL = "<individuals>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))